In [ ]:
# NLU 모델 테스트 노트북
import os
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from transformers import AutoTokenizer, AutoConfig
import commentjson  # 주석이 포함된 JSON 파일을 위해 필요
from torch.nn import functional as F
from torchcrf import CRF
import torch.nn as nn
from transformers import PreTrainedModel
from transformers import AutoModel


# --- 모델 정의 (원본 코드에서 가져옴) ---
class ImprovedRobertaForJointIntentAndNER(PreTrainedModel):
    def __init__(self, config, intent_label_to_id, ner_label_to_id):
        super().__init__(config)
        self.num_intent_labels = len(intent_label_to_id)
        self.num_ner_labels = len(ner_label_to_id)

        # 기본 모델 로드
        from transformers import RobertaModel
        self.roberta = RobertaModel(config)  # 직접 RobertaModel을 사용

        # Intent 분류를 위한 헤드 (개선: 더 복잡한 분류기)
        self.intent_dropout = nn.Dropout(0.2)  # dropout 비율 조정
        self.intent_classifier = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size // 2),
            nn.GELU(),  # ReLU 대신 GELU 사용
            nn.LayerNorm(config.hidden_size // 2),  # Layer Normalization 추가
            nn.Dropout(0.1),
            nn.Linear(config.hidden_size // 2, self.num_intent_labels)
        )

        # NER을 위한 헤드 (개선: BiLSTM + CRF)
        self.ner_dropout = nn.Dropout(0.3)  # NER dropout 증가
        self.ner_lstm = nn.LSTM(
            config.hidden_size,
            config.hidden_size // 2,
            num_layers=2,  # LSTM 레이어 추가
            bidirectional=True,
            batch_first=True,
            dropout=0.2
        )
        self.ner_classifier = nn.Linear(config.hidden_size, self.num_ner_labels)

        # CRF 레이어 (NER용)
        self.crf = CRF(self.num_ner_labels, batch_first=True)

        # 초기 가중치 설정 (훈련된 모델을 로드하므로 여기서는 필요 없음)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        intent_labels=None,
        ner_labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        # RoBERTa 인코더 실행
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]  # 모든 토큰의 임베딩
        pooled_output = sequence_output[:, 0, :]  # [CLS] 토큰 임베딩 (Intent 용)

        # Intent 분류 (개선된 분류기)
        intent_output = self.intent_dropout(pooled_output)
        intent_logits = self.intent_classifier(intent_output)

        # NER 분류 (BiLSTM + CRF)
        ner_output = self.ner_dropout(sequence_output)
        ner_lstm_output, _ = self.ner_lstm(ner_output)
        ner_logits = self.ner_classifier(ner_lstm_output)

        # 예측 단계
        intent_predictions = None
        ner_predictions = None

        if intent_labels is None:
            intent_predictions = torch.argmax(intent_logits, dim=1)

        if ner_labels is None:
            if attention_mask is not None:
                mask = attention_mask.bool()
                best_ner_tags_list = self.crf.decode(ner_logits, mask=mask)

                # 리스트를 텐서로 변환
                ner_predictions = torch.zeros_like(input_ids)
                for i, tags in enumerate(best_ner_tags_list):
                    ner_predictions[i, :len(tags)] = torch.tensor(tags, device=ner_predictions.device)
            else:
                best_ner_tags_list = self.crf.decode(ner_logits)
                ner_predictions = torch.tensor(best_ner_tags_list, device=ner_logits.device)

        return {
            "intent_logits": intent_logits,
            "ner_logits": ner_logits,
            "intent_predictions": intent_predictions,
            "ner_predictions": ner_predictions
        }

# --- 모델 로드 및 테스트를 위한 클래스 ---
class NLUTester:
    def __init__(self, model_dir="./models/integrated_nlu_improved"):
        self.model_dir = model_dir
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"사용 중인 디바이스: {self.device}")
        
        # 레이블 정보 로드
        with open(os.path.join(model_dir, "nlu_labels.jsonc"), 'r', encoding='utf-8') as f:
            self.label_info = commentjson.load(f)
        
        self.intent_id2label = self.label_info["intent_id2label"]
        self.intent_label2id = self.label_info["intent_label2id"]
        self.ner_id2label = self.label_info["ner_id2label"]
        self.ner_label2id = self.label_info["ner_label2id"]
        
        # ID를 문자열에서 정수로 변환 (JSON에서는 키가 문자열로 저장됨)
        self.intent_id2label = {int(k): v for k, v in self.intent_id2label.items()}
        self.ner_id2label = {int(k): v for k, v in self.ner_id2label.items()}
        
        print(f"인텐트 레이블 수: {len(self.intent_id2label)}")
        print(f"NER 레이블 수: {len(self.ner_id2label)}")
        
        # 토크나이저 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        
        # 설정 파일 직접 열고 RoBERTa 설정으로 변환
        try:
            config_file = os.path.join(model_dir, "config.json")
            with open(config_file, 'r', encoding='utf-8') as f:
                config_dict = json.load(f)
            
            # 모델 타입을 roberta로 변경
            config_dict["model_type"] = "roberta"
            
            # 변경된 설정으로 AutoConfig 생성
            from transformers import RobertaConfig
            self.config = RobertaConfig.from_dict(config_dict)
        except Exception as e:
            print(f"설정 파일 변경 중 오류: {str(e)}")
            # 폴백: 기본 RoBERTa 설정 사용
            from transformers import RobertaConfig
            self.config = RobertaConfig.from_pretrained("klue/roberta-base")
            # 레이블 수 설정
            self.config.num_labels = len(self.intent_label2id)
        
        # 직접 모델 생성
        self.model = ImprovedRobertaForJointIntentAndNER(
            self.config, 
            self.intent_label2id, 
            self.ner_label2id
        )
        
        # 모델 가중치 로드
        state_dict = torch.load(os.path.join(model_dir, "pytorch_model.bin"), map_location=self.device)
        
        # 가중치 로드 시 오류 처리 (키 이름 불일치 문제 해결)
        try:
            self.model.load_state_dict(state_dict)
        except Exception as e:
            print(f"가중치 로드 중 오류 발생: {str(e)}")
            print("가중치 로드 방식을 변경하여 다시 시도합니다...")
            
            # 불일치하는 키 수정하여 로드
            from collections import OrderedDict
            new_state_dict = OrderedDict()
            
            for k, v in state_dict.items():
                # 키 이름 변환이 필요한 경우 여기서 처리
                if k.startswith('roberta.'):
                    new_state_dict[k] = v
                else:
                    new_state_dict[k] = v
            
            # 엄격하지 않은 로드 시도
            self.model.load_state_dict(new_state_dict, strict=False)
            print("유연한 방식으로 가중치 로드 완료!")
        
        self.model.to(self.device)
        self.model.eval()
        print("모델 로드 완료!")
    
    def predict(self, text):
        # 입력 텍스트 토큰화
        inputs = self.tokenizer(
            text,
            add_special_tokens=True,
            return_offsets_mapping=True,
            return_tensors="pt",
            truncation=True,
            max_length=256,
            padding="max_length"
        )
        
        input_ids = inputs["input_ids"].to(self.device)
        attention_mask = inputs["attention_mask"].to(self.device)
        offset_mapping = inputs["offset_mapping"][0].numpy()  # 토큰-문자 매핑 정보
        
        # 예측 실행
        with torch.no_grad():
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
        
        # 인텐트 예측 결과 처리
        intent_logits = outputs["intent_logits"][0]
        intent_probs = F.softmax(intent_logits, dim=0).cpu().numpy()
        intent_pred_id = outputs["intent_predictions"][0].item()
        intent_pred = self.intent_id2label[intent_pred_id]
        
        # 인텐트 상위 5개 확률 추출
        top_intent_indices = np.argsort(intent_probs)[::-1][:5]  # 상위 5개
        top_intents = [(self.intent_id2label[idx], intent_probs[idx]) for idx in top_intent_indices]
        
        # NER 예측 결과 처리
        ner_predictions = outputs["ner_predictions"][0].cpu().numpy()
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids[0])
        
        # 토큰별 NER 태그 추출
        ner_tags = []
        entity_spans = []
        current_entity = None
        
        for i, (token_id, pred_id, (start, end)) in enumerate(zip(input_ids[0].cpu().numpy(), 
                                                               ner_predictions, 
                                                               offset_mapping)):
            # 특수 토큰 또는 패딩 건너뛰기
            if start == 0 and end == 0:
                continue
                
            # 예측된 태그 가져오기
            tag = self.ner_id2label.get(pred_id, "O")
            token = self.tokenizer.convert_ids_to_tokens([token_id])[0]
            
            # NER 태그-토큰 쌍 저장
            if tag != "O" and token.startswith("##"):
                # 서브워드 토큰인 경우 이전 토큰에 병합
                if ner_tags:
                    ner_tags[-1] = (ner_tags[-1][0], ner_tags[-1][1] + token[2:], ner_tags[-1][2])
            else:
                ner_tags.append((tag, token, (start, end)))
            
            # 엔티티 추출 (B-tag로 시작하는 경우)
            if tag.startswith("B-"):
                if current_entity:
                    entity_spans.append(current_entity)
                entity_type = tag[2:]
                current_entity = {
                    "entity_type": entity_type,
                    "start": start,
                    "end": end,
                    "text": text[start:end]
                }
            # I-tag인 경우 현재 엔티티에 추가
            elif tag.startswith("I-") and current_entity:
                if tag[2:] == current_entity["entity_type"]:
                    current_entity["end"] = end
                    current_entity["text"] = text[current_entity["start"]:end]
            # O-tag인 경우 현재 엔티티 저장
            elif tag == "O" and current_entity:
                entity_spans.append(current_entity)
                current_entity = None
        
        # 마지막 엔티티 저장
        if current_entity:
            entity_spans.append(current_entity)
        
        # 결과 반환
        result = {
            "text": text,
            "intent": {
                "predicted": intent_pred,
                "confidence": float(intent_probs[intent_pred_id]),
                "top_k": top_intents
            },
            "ner": {
                "token_tags": ner_tags,
                "entities": entity_spans
            }
        }
        
        return result

    def visualize_text_with_entities(self, text, entities):
        """엔티티를 하이라이트하여 HTML로 보여주는 시각화 함수"""
        # 엔티티 색상 맵 정의
        color_map = {
            "PERSON": "#FFADAD",  # 연한 빨강
            "LOCATION": "#FFD6A5",  # 연한 주황
            "ORGANIZATION": "#FDFFB6",  # 연한 노랑
            "DATE": "#CAFFBF", # 연한 초록
            "TIME": "#98F6FF", # 연한 청록
            "MONEY": "#A0C4FF", # 연한 파랑
            "PERCENT": "#BDB2FF", # 연한 남색
            "PHONE": "#FFC6FF", # 연한 자주
            "EMAIL": "#D1D1D1",  # 연한 회색
            "NUMBER": "#99FFFF",  # 연한 하늘색
            "EVENT": "#EBFF99", # 연한 라임
            "PRODUCT": "#FF9CEE",  # 연한 분홍
            "DURATION": "#FFD700",  # 골드
            "FACILITY": "#B4F4A8", # 연한 민트
            "QUANTITY": "#C8A2C8", # 연한 보라
            "CARDINAL": "#F0EAD6"  # 크림색
        }
        
        # 기본 색상
        default_color = "#FFE4E1"  # 연한 붉은색
        
        # 텍스트를 HTML로 변환
        html_text = text
        
        # 엔티티 위치 기준으로 정렬 (뒤에서부터 처리하여 인덱스 문제 방지)
        sorted_entities = sorted(entities, key=lambda x: x["start"], reverse=True)
        
        for entity in sorted_entities:
            entity_type = entity["entity_type"]
            start = entity["start"]
            end = entity["end"]
            entity_text = entity["text"]
            
            # 엔티티 색상 설정
            color = color_map.get(entity_type, default_color)
            
            # HTML 태그로 감싸기
            html_tag = f'<span style="background-color: {color}; padding: 2px; border-radius: 3px;" title="{entity_type}">{entity_text}</span>'
            html_text = html_text[:start] + html_tag + html_text[end:]
        
        return html_text

# --- 인터랙티브 테스트 위젯 생성 ---
def create_test_widgets(tester):
    # 입력 텍스트 위젯
    text_input = widgets.Textarea(
        value='안녕하세요, 내일 서울에서 오후 2시에 김영희 씨와 약속이 있어요.',
        placeholder='테스트할 텍스트를 입력하세요',
        description='입력 텍스트:',
        disabled=False,
        layout=widgets.Layout(width='90%', height='100px')
    )
    
    # 예측 버튼
    predict_button = widgets.Button(
        description='예측하기',
        button_style='primary',
        tooltip='모델로 예측 실행',
        icon='check'
    )
    
    # 출력 영역
    output = widgets.Output()
    
    # 예시 텍스트 버튼
    example_texts = [
        '내일 오후 3시에 홍길동 씨에게 전화해야 해요.',
        '서울에서 부산까지 KTX 표를 예약하고 싶어요.',
        '주문한 상품이 언제 배송될지 알려주세요.',
        '오늘 날씨가 어때요?',
        '내 계좌에서 50만원을 이체해줘.',
        'ABC123 상품의 재고가 있나요?'
    ]
    
    example_buttons = [widgets.Button(description=f'예시 {i+1}', 
                                     layout=widgets.Layout(width='100px')) 
                      for i in range(len(example_texts))]
    
    # 예시 버튼 클릭 이벤트 처리
    def on_example_click(b):
        index = example_buttons.index(b)
        text_input.value = example_texts[index]
    
    for button in example_buttons:
        button.on_click(on_example_click)
    
    # 버튼 클릭 이벤트 처리
    def on_predict_button_clicked(b):
        with output:
            clear_output(wait=True)
            
            if not text_input.value.strip():
                print("입력 텍스트를 입력해주세요!")
                return
            
            try:
                # 모델 예측 실행
                result = tester.predict(text_input.value)
                
                # 결과 시각화
                # 1. 인텐트 결과
                print("📋 분석 결과:")
                print(f"원문: {result['text']}")
                print("\n🎯 인텐트 분석:")
                print(f"예측된 인텐트: {result['intent']['predicted']} (확률: {result['intent']['confidence']:.4f})")
                
                # 2. 인텐트 확률 시각화 (상위 5개)
                plt.figure(figsize=(10, 3))
                intent_names = [intent[0] for intent in result['intent']['top_k']]
                intent_probs = [intent[1] for intent in result['intent']['top_k']]
                
                # 가로 막대 그래프로 표시
                sns.barplot(x=intent_probs, y=intent_names, palette="Blues_r")
                plt.xlim(0, 1)
                plt.title("인텐트 예측 확률 (상위 5개)")
                plt.xlabel("확률")
                plt.ylabel("인텐트")
                plt.tight_layout()
                plt.show()
                
                # 3. NER 결과
                print("\n🔍 개체명(NER) 분석:")
                
                if result['ner']['entities']:
                    # 엔티티 시각화
                    html_text = tester.visualize_text_with_entities(result['text'], result['ner']['entities'])
                    display(HTML(f"<div style='font-size: 16px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;'>{html_text}</div>"))
                    
                    # 인식된 엔티티 표로 출력
                    print("\n인식된 개체명:")
                    for i, entity in enumerate(result['ner']['entities']):
                        print(f"  {i+1}. {entity['text']} ({entity['entity_type']}, 위치: {entity['start']}~{entity['end']})")
                else:
                    print("인식된 개체명이 없습니다.")
                
            except Exception as e:
                print(f"오류 발생: {str(e)}")
                import traceback
                traceback.print_exc()
    
    # 버튼 클릭 이벤트 연결
    predict_button.on_click(on_predict_button_clicked)
    
    # 위젯 배치
    example_box = widgets.HBox(example_buttons)
    
    # 전체 UI 구성
    ui = widgets.VBox([
        widgets.HTML(value="<h2>통합 NLU 모델 테스트</h2>"),
        widgets.HTML(value="<p>아래에 테스트할 텍스트를 입력하거나 예시 버튼을 클릭하세요.</p>"),
        example_box,
        text_input,
        predict_button,
        output
    ])
    
    return ui

# --- 메인 실행 코드 ---
def run_tester():
    try:
        # 기본 모델 디렉토리 설정
        model_dir = "./models/integrated_nlu_improved"
        
        # 모델 디렉토리가 존재하는지 확인
        if not os.path.exists(model_dir):
            print(f"경고: 모델 디렉토리 {model_dir}가 존재하지 않습니다.")
            model_dir = input("모델 디렉토리 경로를 입력해주세요: ")
            if not os.path.exists(model_dir):
                print(f"오류: 모델 디렉토리 {model_dir}가 존재하지 않습니다. 정확한 경로를 확인해주세요.")
                return
        
        # NLU 테스터 초기화
        tester = NLUTester(model_dir)
        
        # 인터랙티브 위젯 생성 및 표시
        ui = create_test_widgets(tester)
        display(ui)
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()

# 노트북에서 실행
if __name__ == "__main__":
    run_tester()

사용 중인 디바이스: cuda
인텐트 레이블 수: 40
NER 레이블 수: 51
모델 로드 완료!
